In [25]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import simplejson as json
import requests
import time
from IPython.display import display

In [26]:
URI = "dbikes1.cmd8vuwgew1e.us-east-1.rds.amazonaws.com"
PORT = "3306"
DB = "dbikes1"
USER = "admin"
PASSWORD = "bikesarefun"

In [27]:
engine = create_engine("mysql+mysqldb://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

In [28]:
sql = "CREATE DATABASE IF NOT EXISTS bike_data"
engine.execute(sql)

2022-02-15 09:36:21,852 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2022-02-15 09:36:21,854 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-15 09:36:21,949 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2022-02-15 09:36:21,950 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ()
2022-02-15 09:36:22,147 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-02-15 09:36:22,148 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-15 09:36:22,434 INFO sqlalchemy.engine.Engine CREATE DATABASE IF NOT EXISTS bike_data
2022-02-15 09:36:22,435 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-15 09:36:22,549 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
for res in engine.execute("SHow Variables;"):
    print(res)

2022-02-15 09:36:22,784 INFO sqlalchemy.engine.Engine SHow Variables;
2022-02-15 09:36:22,785 INFO sqlalchemy.engine.Engine [raw sql] ()
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('admin_ssl_ca', '')
('admin_ssl_capath', '')
('admin_ssl_cert', '')
('admin_ssl_cipher', '')
('admin_ssl_crl', '')
('admin_ssl_crlpath', '')
('admin_ssl_key', '')
('admin_tls_ciphersuites', '')
('admin_tls_version', 'TLSv1,TLSv1.1,TLSv1.2,TLSv1.3')
('authentication_policy', '*')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment_offset', '1')
('autocommit', 'OFF')
('automatic_sp_privileges', 'ON')
('avoid_temporal_upgrade', 'OFF')
('back_log', '65')
('basedir', '/rdsdbbin/mysql-8.0.27.R3/')
('big_tables', 'OFF')
('bind_address', '*')
('binlog_cache_size', '32768')
('binlog_checksum', 'CRC32')
('binlog_direct_non_transactional_updates', 'OFF')
('binlog_encryption', 'OFF')
('binlog_error_action', 'ABORT_SERVER')
('binlog_expire_logs_second

In [30]:
sql2 = """
CREATE TABLE IF NOT EXISTS station (
number INTEGER,
contract_name VARCHAR(256),
name VARCHAR(256),
address VARCHAR(256),
position_lat REAL,
position_long REAL,
banking VARCHAR(256),
bike_stands INTEGER,
available_bike_stands INTEGER,
available_bikes INTEGER,
status VARCHAR(256),
last_update INTEGER
)

"""
try:
    res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql2)
    print(res.fetchall())
except Exception as e:
    print(e)

2022-02-15 09:36:23,141 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS station
2022-02-15 09:36:23,142 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-15 09:36:23,244 INFO sqlalchemy.engine.Engine COMMIT
2022-02-15 09:36:23,434 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS station (
address VARCHAR(256),
banking INTEGER,
bike_stands INTEGER,
bonus INTEGER,
contract_name VARCHAR(256),
name VARCHAR(256),
number INTEGER,
position_lat REAL,
position_long REAL,
status VARCHAR(256)
)


2022-02-15 09:36:23,437 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-15 09:36:23,569 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [31]:
sql3 = """
CREATE TABLE IF NOT EXISTS available (
number INTEGER,
available_bike_stands INTEGER,
available_bikes INTEGER,
status VARCHAR(256),
last_update INTEGER

)

"""
try:
    res = engine.execute(sql3)
    print(res.fetchall())
except Exception as e:
    print(e)

2022-02-15 09:36:23,789 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS available (
number INTEGER,
available_bike_stands INTEGER,
available_bikes INTEGER,
status VARCHAR(256),
last_update INTEGER

)


2022-02-15 09:36:23,789 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-15 09:36:23,909 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [32]:
import requests
r = requests.get("https://api.jcdecaux.com/vls/v1/stations?contract=dublin&apiKey=0d101d6645f441b34490f9530a0f91c869debe77")
print(r)
r.json()

<Response [200]>


[{'number': 42,
  'contract_name': 'dublin',
  'name': 'SMITHFIELD NORTH',
  'address': 'Smithfield North',
  'position': {'lat': 53.349562, 'lng': -6.278198},
  'banking': False,
  'bonus': False,
  'bike_stands': 30,
  'available_bike_stands': 9,
  'available_bikes': 21,
  'status': 'OPEN',
  'last_update': 1644917309000},
 {'number': 30,
  'contract_name': 'dublin',
  'name': 'PARNELL SQUARE NORTH',
  'address': 'Parnell Square North',
  'position': {'lat': 53.353462, 'lng': -6.265305},
  'banking': False,
  'bonus': False,
  'bike_stands': 20,
  'available_bike_stands': 5,
  'available_bikes': 15,
  'status': 'OPEN',
  'last_update': 1644917675000},
 {'number': 54,
  'contract_name': 'dublin',
  'name': 'CLONMEL STREET',
  'address': 'Clonmel Street',
  'position': {'lat': 53.336021, 'lng': -6.26298},
  'banking': False,
  'bonus': False,
  'bike_stands': 33,
  'available_bike_stands': 22,
  'available_bikes': 11,
  'status': 'OPEN',
  'last_update': 1644917587000},
 {'number': 108

In [33]:

import traceback
import datetime
import time


def write_to_file(text):
    with open ("data/bikes_{}".format(now).replace(" ", "_", "w")) as f:
        f.write(r.text)

def write_to_db(text):

    def main():
        while True:
            try:
                now = datetime.datetime.now()
                r = requests.get(STATIONS_URI, params={"apikey":APIKEY, "contract": NAME })
                print(r,now)
                write_to_file(r.text)
                write_to_db(r.text)
                time.sleep(5*60)
            except:
                print(traceback.format_exc())
                if engine is None:
                    return

In [34]:
r.json()

[{'number': 42,
  'contract_name': 'dublin',
  'name': 'SMITHFIELD NORTH',
  'address': 'Smithfield North',
  'position': {'lat': 53.349562, 'lng': -6.278198},
  'banking': False,
  'bonus': False,
  'bike_stands': 30,
  'available_bike_stands': 9,
  'available_bikes': 21,
  'status': 'OPEN',
  'last_update': 1644917309000},
 {'number': 30,
  'contract_name': 'dublin',
  'name': 'PARNELL SQUARE NORTH',
  'address': 'Parnell Square North',
  'position': {'lat': 53.353462, 'lng': -6.265305},
  'banking': False,
  'bonus': False,
  'bike_stands': 20,
  'available_bike_stands': 5,
  'available_bikes': 15,
  'status': 'OPEN',
  'last_update': 1644917675000},
 {'number': 54,
  'contract_name': 'dublin',
  'name': 'CLONMEL STREET',
  'address': 'Clonmel Street',
  'position': {'lat': 53.336021, 'lng': -6.26298},
  'banking': False,
  'bonus': False,
  'bike_stands': 33,
  'available_bike_stands': 22,
  'available_bikes': 11,
  'status': 'OPEN',
  'last_update': 1644917587000},
 {'number': 108

In [36]:
def stations_to_db(text):
    stations = json.loads(text)
    print(stations)
    print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (station.get("address"),int(station.get("banking")),
               station.get("bike_stands"),(station.get("address")),
                station.get("contract_name"),station.get("name"),
                station.get("number"),station.get("position").get("lat"),
               station.get("position").get("lng"), station.get("status")
               )
        engine.execute("insert into station values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",vals)
        break
    return
stations_to_db(r.text)
# abc

[{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 9, 'available_bikes': 21, 'status': 'OPEN', 'last_update': 1644917309000}, {'number': 30, 'contract_name': 'dublin', 'name': 'PARNELL SQUARE NORTH', 'address': 'Parnell Square North', 'position': {'lat': 53.353462, 'lng': -6.265305}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 5, 'available_bikes': 15, 'status': 'OPEN', 'last_update': 1644917675000}, {'number': 54, 'contract_name': 'dublin', 'name': 'CLONMEL STREET', 'address': 'Clonmel Street', 'position': {'lat': 53.336021, 'lng': -6.26298}, 'banking': False, 'bonus': False, 'bike_stands': 33, 'available_bike_stands': 22, 'available_bikes': 11, 'status': 'OPEN', 'last_update': 1644917587000}, {'number': 108, 'contract_name': 'dublin', 'name': 'AVONDALE ROAD', 'address': 'Avo